# Run Quarterly

In [7]:
from database.market import Market
import pandas as pd
from tqdm import tqdm
from datetime import datetime,timezone, timedelta
import pandas_datareader as pdr
from extractor.finnhub_extractor import FinnhubExtractor
from extractor.tiingo_extractor import TiingoExtractor
from time import sleep

In [11]:
start = (datetime.now() - timedelta(days=730)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
market = Market()

In [3]:
sp5 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]

In [10]:
market.connect()
market.drop_table("sp500")
market.store_data("sp500",sp5)
market.close()

In [5]:
market.connect()
tickers = market.retrieve_data("sp500").sort_values("Symbol")["Symbol"]
market.close()

In [12]:
market.connect()
market.drop_table("portfolio_pdr_prices")
market.drop_table("portfolio_tiingo_prices")
market.drop_table("portfolio_finnhub_prices")
fails = []
for ticker in tqdm(tickers):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        status = "tiingo"
        price = pdr.DataReader(ticker, 
                       start=start, 
                       end=end, 
                       data_source='yahoo')
        price["ticker"] = ticker
        market.store_data("portfolio_pdr_prices",price.reset_index())
        tp = TiingoExtractor(ticker,start,end).extract()
        tp["ticker"] = ticker
        market.store_data("portfolio_tiingo_prices",tp.reset_index())
        fp = FinnhubExtractor.extract(ticker,start,end)
        fp = pd.DataFrame(fp)
        fp["ticker"] = ticker
        market.store_data("portfolio_finnhub_prices",fp.reset_index())
        sleep(8)
    except Exception as e:
        msg = {"status":status,"ticker":ticker,"exception":str(e)}
        print(msg,ticker)
        fails.append(ticker)
market.close()

 22%|██████████████████████████████████████▎                                                                                                                                    | 113/505 [16:43<42:01,  6.43s/it]

Expecting value: line 1 column 1 (char 0)
{'status': 'tiingo', 'ticker': 'CRL', 'exception': "'NoneType' object does not support item assignment"} CRL


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 501/505 [1:14:24<00:25,  6.45s/it]

Expecting value: line 1 column 1 (char 0)
{'status': 'tiingo', 'ticker': 'YUM', 'exception': "'NoneType' object does not support item assignment"} YUM


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 502/505 [1:14:24<00:13,  4.66s/it]

...

Expecting value: line 1 column 1 (char 0)
{'status': 'tiingo', 'ticker': 'ZBRA', 'exception': "'NoneType' object does not support item assignment"} ZBRA


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 504/505 [1:14:26<00:02,  2.57s/it]

Expecting value: line 1 column 1 (char 0)
{'status': 'tiingo', 'ticker': 'ZION', 'exception': "'NoneType' object does not support item assignment"} ZION


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [1:14:26<00:00,  8.85s/it]

Expecting value: line 1 column 1 (char 0)
{'status': 'tiingo', 'ticker': 'ZTS', 'exception': "'NoneType' object does not support item assignment"} ZTS
